# RNS Patient Electrode Colocation

In [5]:
import os
import pandas as pd
import scipy.io as sio
from scipy.spatial import distance

from get_iEEG_data import *
from iEEG_helper_functions import *

In [6]:
RNS_DIR = "../../cnt-data/rns_patients.mat"
MAX_DIST = 9  # millimeters

In [7]:
# Load mat file
rns_data = sio.loadmat(RNS_DIR)
rns_data = rns_data["patients_Penn"][0]

In [8]:
rns_hup_ids = []

for row in rns_data:
    patient_id = row[0][0]
    institution = patient_id[:3]
    if institution != "HUP":
        continue
    hup_id = patient_id[3:]

    ieeg_electrodes, ieeg_coordinates = row[2], row[3]
    rns_electrodes, rns_coordinates = row[10], row[13]
    # If any of ieeg_electrodes, ieeg_coordinates, rns_electrodes, rns_coordinates are empty, skip
    if (
        len(ieeg_electrodes[0]) == 0
        or len(ieeg_coordinates[0]) == 0
        or len(rns_electrodes[0]) == 0
        or len(rns_coordinates[0]) == 0
    ):
        # print(f"HUP {hup_id} has empty iEEG or RNS electrodes, skip...")
        continue

    print(f"HUP {hup_id}")
    rns_hup_ids.append(int(hup_id))
    # Calculate the pairwise distances between iEEG and RNS electrodes
    distances = distance.cdist(ieeg_coordinates, rns_coordinates, metric="euclidean")

    # For each RNS electrode, find the closest iEEG electrode within 5mm
    for j, rns_electrode in enumerate(rns_electrodes):
        closest_ieeg_distance = float("inf")  # start with a big number
        closest_ieeg_electrode = None
        for i, ieeg_distance in enumerate(
            distances[:, j]
        ):  # for each distance to the RNS electrode
            if (
                ieeg_distance < closest_ieeg_distance and ieeg_distance < MAX_DIST
            ):  # within 5mm
                closest_ieeg_distance = ieeg_distance
                closest_ieeg_electrode = ieeg_electrodes[i]

        if closest_ieeg_electrode:
            print(
                f"RNS electrode {rns_electrode[0][0]} is closest to iEEG electrode {closest_ieeg_electrode[0][0]} at {closest_ieeg_distance:.2f} mm."
            )
        else:
            print(
                f"RNS electrode {rns_electrode[0][0]} has no iEEG electrode within {MAX_DIST}mm."
            )

HUP 084
RNS electrode CAN1 has no iEEG electrode within 9mm.
RNS electrode CAN2 has no iEEG electrode within 9mm.
RNS electrode CAN3 has no iEEG electrode within 9mm.
RNS electrode CAN4 has no iEEG electrode within 9mm.
RNS electrode LH1 is closest to iEEG electrode LMST1 at 5.90 mm.
RNS electrode LH2 is closest to iEEG electrode LH2 at 6.94 mm.
RNS electrode LH3 is closest to iEEG electrode LH1 at 3.60 mm.
RNS electrode LH4 has no iEEG electrode within 9mm.
RNS electrode RH1 has no iEEG electrode within 9mm.
RNS electrode RH2 is closest to iEEG electrode RA1 at 7.17 mm.
RNS electrode RH3 is closest to iEEG electrode RA1 at 7.63 mm.
RNS electrode RH4 has no iEEG electrode within 9mm.
HUP 101
RNS electrode CAN1 has no iEEG electrode within 9mm.
RNS electrode CAN2 has no iEEG electrode within 9mm.
RNS electrode CAN3 has no iEEG electrode within 9mm.
RNS electrode CAN4 has no iEEG electrode within 9mm.
RNS electrode LSIH1 has no iEEG electrode within 9mm.
RNS electrode LSIH2 has no iEEG e

In [9]:
print(f"Total number of HUP/RNS patients: {len(rns_hup_ids)}")

Total number of HUP/RNS patients: 20


In [10]:
rns_hup_ids

[84,
 101,
 108,
 119,
 121,
 127,
 128,
 129,
 131,
 136,
 137,
 143,
 147,
 153,
 156,
 159,
 182,
 192,
 197,
 199]

In [11]:
# Load HUP_implant_dates.xlsx
nina_patients_df = pd.read_excel("../../cnt-data/HUP_implant_dates.xlsx")
# Make the hup_id column integers
nina_patients_df["hup_id"] = nina_patients_df["hup_id"].astype(int)
nina_patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


In [12]:
# Keep only the rows with hup_id in rns_hup_ids
nina_patients_df = nina_patients_df[nina_patients_df["hup_id"].isin(rns_hup_ids)]
# Reset the index
nina_patients_df = nina_patients_df.reset_index(drop=True)
nina_patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,199,HUP199_phaseII,2019-11-13,07:15:00,2019-11-20 00:00:00,64.9
1,197,HUP197_phaseII_D01-02,2019-10-28,07:15:00,2019-11-15 00:00:00,88.8
2,192,HUP192_phaseII,2019-04-24,07:15:00,2019-05-03 15:04:00,49.4
3,182,HUP182_phaseII,2018-12-12,07:15:00,2018-12-24 00:00:00,55.8
4,159,HUP159_phaseII_D01-D02,2018-01-10,07:15:00,2018-01-24 00:00:00,93.4
5,156,HUP156_phaseII_D01-D03,2017-12-13,07:15:00,2017-12-22 00:00:00,57.6
6,153,HUP153_phaseII_D01-D02,2017-11-08,07:15:00,2017-11-21 00:00:00,101.6
7,147,HUP147_phaseII_D01-D02,2017-08-28,07:15:00,2017-09-05 22:07:00,96.6
8,143,HUP143_phaseII,2017-07-17,07:15:00,2017-07-29 00:00:00,61.5
9,137,HUP137_phaseII_D01-D03,2017-03-22,07:15:00,2017-03-29 13:30:00,141.3


In [13]:
rns_nina_hup_ids = nina_patients_df["hup_id"].tolist()
rns_nina_hup_ids

[199, 197, 192, 182, 159, 156, 153, 147, 143, 137]

In [14]:
# def get_patient_hup_ids(directory):
#     # List all files in the directory
#     files = os.listdir(directory)

#     # Filter out files based on the given pattern and extract patient_hup_id as integers
#     patient_hup_ids = [
#         int(f.split("_")[1].split(".")[0])
#         for f in files
#         if f.startswith("HUP_") and f.endswith(".npy")
#     ]

#     return patient_hup_ids


# directory = "../../cnt-data/synchrony/all/broadband/"
# completed_hup_ids = get_patient_hup_ids(directory)
# completed_hup_ids.sort()
# print(completed_hup_ids)

In [15]:
# # Find the intersection of completed_hup_ids and rns_nina_hup_ids
# rns_nina_completed_hup_ids = list(set(completed_hup_ids) & set(rns_nina_hup_ids))
# rns_nina_completed_hup_ids.sort()
# rns_nina_completed_hup_ids

In [16]:
# Create an empty list to store hup_id and corresponding ieeg electrodes
data = []

for row in rns_data:
    patient_id = row[0][0]
    institution = patient_id[:3]
    if institution != "HUP":
        continue
    hup_id = patient_id[3:]

    # if int(hup_id) not in rns_nina_completed_hup_ids:
    if int(hup_id) not in rns_nina_hup_ids:
        continue

    ieeg_electrodes, ieeg_coordinates = row[2], row[3]
    rns_electrodes, rns_coordinates = row[10], row[13]

    # If any of the fields are empty, skip
    if not all(
        len(field[0])
        for field in [
            ieeg_electrodes,
            ieeg_coordinates,
            rns_electrodes,
            rns_coordinates,
        ]
    ):
        continue

    # Calculate the pairwise distances between iEEG and RNS electrodes
    distances = distance.cdist(ieeg_coordinates, rns_coordinates, metric="euclidean")

    # For each RNS electrode, find the closest iEEG electrode within 5mm
    closest_ieeg_electrodes = []
    for j, rns_electrode in enumerate(rns_electrodes):
        closest_ieeg_distance = float("inf")
        closest_ieeg_electrode = None
        for i, ieeg_distance in enumerate(distances[:, j]):
            if ieeg_distance < closest_ieeg_distance and ieeg_distance < MAX_DIST:
                closest_ieeg_distance = ieeg_distance
                closest_ieeg_electrode = ieeg_electrodes[i]

        if closest_ieeg_electrode:
            closest_ieeg_electrodes.append(closest_ieeg_electrode[0][0])

    if len(closest_ieeg_electrodes) > 2:
        # Append the hup_id and unique closest_ieeg_electrodes (sorted) to the data list
        data.append([int(hup_id), sorted(list(set(closest_ieeg_electrodes)))])

# Convert the data list into a pandas DataFrame
df = pd.DataFrame(data, columns=["hup_id", "electrodes"])
df

,hup_id,electrodes
0,143,"[LA2, LA3, LB3, RA1, RA2, RB1, RB2]"
1,147,"[LA2, LA3, LB2, LD1, LD2, LD3, LD5, LK1, LL8, ..."
2,153,"[LA3, LB1, LC1, LJ1, LJ2, RA2, RB1, RB3, RE12]"
3,159,"[LD1, LD2, LF8, LH1]"
4,182,"[LG2, RB1, RB3, RC2]"
5,197,"[LB6, LB7, LC3, LG1, LG2, LG3]"
6,199,"[LA1, LB1, LC1, RB2, RB3]"


In [17]:
# Save df into a csv file in ../../cnt-data/
df.to_csv(f"../../cnt-data/HUP_RNS_electrodes.csv", index=False)

In [20]:
# Get the list of electrodes for hup_id = 147
df[df["hup_id"] == 197]["electrodes"].tolist()[0]

['LB6', 'LB7', 'LC3', 'LG1', 'LG2', 'LG3']